In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from kerastuner.tuners import RandomSearch

C:\Users\RPAREPAL\AppData\Local\Temp/ipykernel_13136/4269418548.py:21: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
#Importing data
l_input_files=['Churn']

cur_dir=os.getcwd()
data_dir=os.path.join(cur_dir,'data','*')
ls_files=[f  for f in glob.glob(data_dir) \
          if (Path(f).stem in l_input_files and os.path.basename(f).endswith('csv')) ]

#print(ls_files)
df=pd.DataFrame()
for f in ls_files:
    print('file to be read: {}'.format(os.path.basename(f)))
    df=pd.read_csv(f)

file to be read: Churn.csv


In [45]:
churn_df=df.copy()

In [47]:
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
X = churn_df.iloc[:, 3:13]
y = churn_df.iloc[:, 13]

In [5]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
# Concatenate dummy varaibles to Data Frames and drop features
X=pd.concat([X,geography,gender],axis=1)
X=X.drop(['Geography','Gender'],axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

## Building Baseline ANN

In [9]:
baseline_model = Sequential()

In [10]:
baseline_model.add(Dense(units = 6, kernel_initializer= 'he_uniform',activation='relu',input_dim = 11))
baseline_model.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
# Adding output layer
baseline_model.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

In [11]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [12]:
baseline_model.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
baseline_model_hist=baseline_model.fit(x_train, y_train,validation_split=0.3, batch_size = 10, epochs = 100)

Epoch 1/100
560/560 [==============================] - 1s 951us/step - loss: 29263.6191 - accuracy: 0.7961 - val_loss: 16435.3379 - val_accuracy: 0.7958
Epoch 2/100
560/560 [==============================] - 0s 741us/step - loss: 10101.3369 - accuracy: 0.7961 - val_loss: 4782.5015 - val_accuracy: 0.7958
Epoch 3/100
560/560 [==============================] - 0s 750us/step - loss: 1936.7596 - accuracy: 0.7241 - val_loss: 104.2366 - val_accuracy: 0.4833
Epoch 4/100
560/560 [==============================] - 0s 773us/step - loss: 54.4334 - accuracy: 0.4768 - val_loss: 27.5955 - val_accuracy: 0.3921
Epoch 5/100
560/560 [==============================] - 0s 777us/step - loss: 23.8008 - accuracy: 0.4184 - val_loss: 28.9950 - val_accuracy: 0.4812
Epoch 6/100
560/560 [==============================] - 1s 1ms/step - loss: 17.1466 - accuracy: 0.4946 - val_loss: 15.8222 - val_accuracy: 0.7362
Epoch 7/100
560/560 [==============================] - 1s 1ms/step - loss: 12.3595 - accuracy: 0.6977 - va

560/560 [==============================] - 1s 1ms/step - loss: 4.3586 - accuracy: 0.7591 - val_loss: 6.7182 - val_accuracy: 0.5579
Epoch 58/100
560/560 [==============================] - 1s 1ms/step - loss: 5.1823 - accuracy: 0.7616 - val_loss: 2.0248 - val_accuracy: 0.6808
Epoch 59/100
560/560 [==============================] - 1s 1ms/step - loss: 5.2293 - accuracy: 0.7511 - val_loss: 5.6590 - val_accuracy: 0.7817
Epoch 60/100
560/560 [==============================] - 1s 1ms/step - loss: 4.1752 - accuracy: 0.7638 - val_loss: 3.8241 - val_accuracy: 0.7950
Epoch 61/100
560/560 [==============================] - 1s 1ms/step - loss: 4.0565 - accuracy: 0.7563 - val_loss: 9.9200 - val_accuracy: 0.7879
Epoch 62/100
560/560 [==============================] - 1s 1ms/step - loss: 5.0754 - accuracy: 0.7532 - val_loss: 2.2814 - val_accuracy: 0.7475
Epoch 63/100
560/560 [==============================] - 1s 1ms/step - loss: 4.1118 - accuracy: 0.7575 - val_loss: 5.2973 - val_accuracy: 0.7962
Epoch

In [14]:
baseline_model.evaluate(x_test,y_test)
#test_loss = model.evaluate(test_images, test_labels)

63/63 [==============================] - 0s 1ms/step - loss: 5.3787 - accuracy: 0.7980


[5.378667831420898, 0.7979999780654907]

### HyperParameter Tunning using KerasTunner

In [15]:
import kerastuner as kt

In [36]:
def model_builder_template(units, activation, dropout, lr):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [37]:
def build_model(hp):
    #for j in range(hp.Int('Num_Layers',2,20)):
        units = hp.Int("Layer", min_value=32, max_value=512, step=32)
        activation = hp.Choice("activation", ["relu", "tanh"])
        dropout = hp.Boolean("dropout")
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
   
        # call existing model-building code with the hyperparameter values.
        model = model_builder_template(
            units=units, activation=activation, dropout=dropout, lr=lr
        )
        return model

In [38]:
tuner = kt.Hyperband(build_model, # the hypermodel
                     objective='val_accuracy', # objective to optimize
max_epochs=10,
factor=3, # factor which you have seen above 
directory='dir', # directory to save logs 
project_name='khyperband')

INFO:tensorflow:Reloading Oracle from existing project dir\khyperband\oracle.json
INFO:tensorflow:Reloading Tuner from dir\khyperband\tuner0.json


In [39]:
# hypertuning settings
tuner.search_space_summary() 

Search space summary
Default search space size: 24
Num_Layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
Layer_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
Layer_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Layer_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Layer_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Layer_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 

In [40]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Perform hypertuning
tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

INFO:tensorflow:Oracle triggered exit


In [41]:
best_hp=tuner.get_best_hyperparameters()[0]

In [43]:
hp = kt.HyperParameters()

In [44]:
hypermodel = build_model(hp)
best_hp = tuner.get_best_hyperparameters()[0]
model = hypermodel.build(best_hp)
#hypermodel.fit(best_hp, model, x_all, y_all, epochs=1)

KeyError: '0 does not exist.'

In [23]:
h_model = tuner.hypermodel.build(best_hp)

In [26]:
h_model = tuner.hypermodel.build(best_hp)

In [28]:
h_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [25]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hp)
h_model.summary()
h_model.fit(x_train, x_test, epochs=10, validation_split=0.2)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
def build_nn(hpt_nn):
    model=keras.Sequential()
    for j in range(hpt_nn.Int('Num_Layers',2,20)):
        model.add(keras.layers.Dense(units=hpt_nn.Int("Layer_"+str(j),
                                                     min_value=32,
                                                     max_value=512,
                                                     step=32), activation='relu'))
    model.add(keras.layers.Dense(units=1,activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(hpt_nn.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                 loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model
                              
                              

In [ ]:
import keras_tuner as kt

#build_model(kt.HyperParameters())

In [ ]:
tun_model=RandomSearch(build_nn,objective='val_mean_absolute_error',
                       max_trials=5,
                       executions_per_trial=3,
                       directory='project',
                       project_name='Churn Model' )

In [ ]:
tun_model.search_space_summary()

In [ ]:
 tun_model.search(X_train, y_train,
             epochs=5,
             validation_data=(x_train, y_train))

In [ ]:
# Get the top 2 models.
models = tun_model.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None,11))
#input_shape=(None, 28, 28)
#best_model.summary()



In [ ]:
best_model_hist=best_model.fit(X_train, y_train,validation_split=0.3, batch_size = 10, epochs = 100)

In [ ]:
best_model_hist.history.keys()

In [ ]:


# summarize history for loss
plt.plot(best_model_hist.history['loss'])
plt.plot(best_model_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
best_hps = tun_model.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_nn(best_hps[0])
# Fit with the entire dataset.


best_model_hist=model.fit(X_train, y_train,validation_split=0.3, batch_size = 10, epochs = 100)

In [ ]:
#best_model.predict(x_test,y_test)

y_pred = best_model.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
y_test_df = pd.DataFrame(y_test)
y_test_df['ID'] = y_test_df.index

y_pred_df.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

y_pred_final = pd.concat([y_test_df,y_pred_df],axis=1)
y_pred_final= y_pred_final.rename(columns={ 0: 'Churn_Prob'})

y_pred_final['predicted'] = y_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.5 else 0)
#y_pred1 = (y_pred > 0.5)

#tun_model.get_best_hyperparameters(5)
best_hps = tun_model.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val)

In [ ]:
confusion = confusion_matrix( y_pred_final.Exited, y_pred_final.predicted )
confusion

In [ ]:
accuracy_score( y_pred_final.Exited, y_pred_final.predicted)

### ROC Curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(6, 4))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return fpr, tpr, thresholds

In [ ]:
draw_roc(y_pred_final.Exited, y_pred_final.predicted)

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_pred_final[i]= y_pred_final.Churn_Prob.map( lambda x: 1 if x > i else 0)
y_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix
num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix( y_pred_final.Exited, y_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    sensi = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    speci = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

#### From the curve above, 0.4 is the optimum point to take it as a cutoff probability.

In [ ]:
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.4 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
accuracy_score( y_pred_final.Exited, y_pred_final.final_predicted)

In [ ]:
confusion_matrix( y_pred_final.Exited, y_pred_final.final_predicted )